In [1]:
import numpy as np
import cv2

In [2]:
image_object = cv2.imread("C:\\Users\\Darshan\\Downloads\\Computer Vision\\Logo.jpg", cv2.IMREAD_GRAYSCALE)
image_scene = cv2.imread("C:\\Users\\Darshan\\Downloads\\Computer Vision\\Bottle.jpg", cv2.IMREAD_GRAYSCALE)

In [3]:
# Use SIFT for feature detection
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(image_object, None)
kp2, des2 = sift.detectAndCompute(image_scene, None)

In [4]:
# Use FLANN for feature matching
index_params = dict(algorithm=1, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

In [5]:
# Use Lowe's Ratio
good_matches = []
for m, n in matches:
  if m.distance < 0.75 * n.distance:
    good_matches.append(m)

In [9]:
# If mathces are good
if(len(good_matches) > 10):
  src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
  dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

  # Find homography
  M, m = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

  h, w = image_object.shape
  pts = np.float32([[0,0], [0,h-1], [w-1,h-1], [w-1,0]]).reshape(-1, 1, 2)
  dst = cv2.perspectiveTransform(pts, M)

  # Draw polylines
  image_result = cv2.polylines(cv2.cvtColor(image_scene, cv2.COLOR_GRAY2BGR), [np.int32(dst)], True, (0, 255, 0), 3, cv2.LINE_AA)
else:
  print("Not enough matches")

In [10]:
# Draw keypoint matches (optional)
draw_params = dict(matchColor=(0, 255, 0), singlePointColor = None, flags=2)
image_match = cv2.drawMatches(image_object, kp1, image_scene, kp2, good_matches, None, **draw_params)

In [11]:
# Show images
cv2.imshow("CocaColaImage",image_match)
cv2.waitKey(0)
cv2.destroyAllWindows()